<a href="https://www.kaggle.com/code/lukalafaye/hub-eau-dataset?scriptVersionId=210664971" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
import requests
import pandas as pd

# Juste un code réseau pour cette api...

url = "https://hubeau.eaufrance.fr/api/v1/ecoulement/observations"
params = {
    "format": "json",
    "code_departement": "75,77,78,91,92,93,94,95",
    "fields": "code_station,libelle_station,code_commune,libelle_commune,latitude,longitude,code_ecoulement,libelle_ecoulement,date_observation",
    "date_observation_min": "2020-01-01",
    "date_observation_max": "2024-12-31"
}
response = requests.get(url, params=params)

if response.status_code == 200:
    data = response.json()["data"]
    
    df_ecoulement = pd.DataFrame(data)
    
    """
    df_ecoulement["code_reseau"] = df_ecoulement["code_reseau"].apply(
        lambda x: x if isinstance(x, list) else [x]
    )
    df_ecoulement_expanded = df_ecoulement.explode("code_reseau")
    """ 
    
    df_ecoulement_expanded=df_ecoulement
    df_ecoulement_expanded.to_csv("ile_de_france_ecoulement_2022-2024.csv", index=False)
    print("Data saved to ile_de_france_ecoulement_2022-2024.csv")
else:
    print(f"Failed to fetch data: {response.status_code} - {response.text}")

df_ecoulement_expanded["code_commune"].unique()

In [ ]:
import requests
import pandas as pd

url = "https://hubeau.eaufrance.fr/api/v1/qualite_eau_potable/resultats_dis"
params = {
    "format": "json",
    "fields": (
        "code_commune,nom_commune,date_prelevement,libelle_parametre,"
        "resultat_numerique,libelle_unite,conclusion_conformite_prelevement"
    ),
    "code_departement": "75,77,78,91,92,93,94,95",
    "date_min_prelevement": "2020-01-01T00:00:00",
    "date_max_prelevement": "2024-12-31T23:59:59",
    "size": 10000,
    "page": 1,
}

all_data = []

while True:
    response = requests.get(url, params=params)
    if response.status_code in [200, 206]:
        response_data = response.json()
        data = response_data.get("data", [])
        all_data.extend(data)
        next_page = response_data.get("next")
        if not next_page:
            break
        params["page"] += 1
        break # only one page
    else:
        print(f"Failed to fetch data on page {params['page']}: {response.status_code} - {response.text}")
        break

df_qualite = pd.DataFrame(all_data)
df_qualite = df_qualite[df_qualite["libelle_unite"] != "SANS OBJET"]
df_qualite["conclusion_conformite_prelevement"] = df_qualite["conclusion_conformite_prelevement"].apply(
    lambda x: 1 if "Eau d'alimentation conforme" in x else 0
)

"""
df_qualite["code_reseau"] = df_qualite["code_reseau"].apply(
    lambda x: x if isinstance(x, list) else [x]
)
df_qualite_expanded = df_qualite.explode("code_reseau")
"""

df_qualite_expanded = df_qualite
df_qualite_expanded.to_csv("ile_de_france_water_quality_2022-2024.csv", index=False)
print("Data saved to ile_de_france_water_quality_2022-2024.csv")

In [ ]:
a = df_qualite_expanded["code_commune"].unique()
b = df_ecoulement_expanded["code_commune"].unique()

def intersection(a, b):
    return [ e for e in a if e in b ]

print(intersection(b, a))

In [ ]:
df_qualite_expanded["code_commune"] = df_qualite_expanded["code_commune"].astype(str)
df_ecoulement_expanded["code_commune"] = df_ecoulement_expanded["code_commune"].astype(str)

df_merged = pd.merge(
    df_qualite_expanded,
    df_ecoulement_expanded,
    on="code_commune",
    how="inner",
    suffixes=("_quality", "_ecoulement")
)

df_merged.to_csv("ile_de_france_merged.csv", index=False)
print("Merged data saved to ile_de_france_merged.csv")

In [ ]:
import requests
import pandas as pd

url_contaminants = "https://hubeau.eaufrance.fr/api/v1/surveillance_littoral/contaminants_chimiques"
params_contaminants = {
    "format": "json",
    "fields": (
        "code_lieusurv,libelle_lieusurv,longitude,latitude,"
        "date_prel,code_parametre,libelle_parametre,"
        "resultat_analyse,libelle_unite_resultat"
    ),
    "date_min_prel": "2022-01-01T00:00:00",
    "date_max_prel": "2024-12-31T23:59:59",
    "size": 500,
    "page": 1,
}

all_contaminants_data = []
current_page = 1

while True:
    response_contaminants = requests.get(url_contaminants, params=params_contaminants)
    if response_contaminants.status_code in [200, 206]:
        response_data = response_contaminants.json()
        data_contaminants = response_data.get("data", [])
        
        if not data_contaminants:
            break
        
        all_contaminants_data.extend(data_contaminants)
        
        next_page = response_data.get("next")
        if not next_page or current_page * params_contaminants["size"] >= 20000:
            break
        
        params_contaminants["page"] += 1
        current_page += 1
    else:
        break

if all_contaminants_data:
    df_contaminants = pd.DataFrame(all_contaminants_data)
    selected_columns = [
        "code_lieusurv", "libelle_lieusurv", "longitude", "latitude", 
        "date_prel", "code_parametre", 
        "libelle_parametre", "resultat_analyse", "libelle_unite_resultat"
    ]
    df_contaminants_filtered = df_contaminants[selected_columns]

    df_contaminants_expanded = df_contaminants_filtered
    df_contaminants_expanded.to_csv("littoral_contaminants_2022-2024.csv", index=False)
    print("Data saved to littoral_contaminants_2022-2024.csv")

In [ ]:
# Merge using coordinates?